In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import time

In [15]:
base_url = "https://www.etoos.com/teacher/Evaluate/default.asp?"
teacher_id_list = [200180, 200439]
page_range = 34
# -> 총 데이터 개수 = 선생 수 x 페이지 범위 x 한 페이지 당 개수 = 2 x 34 x 15 = 1020
teacher_prefix = "TEACHER_ID="
page_prefix = "&page="
path = "./data/"
f_name = 'review.csv'
csv_full_name = path + f_name

def getFullUrl(teacher_id:int, page:int)->str:
    global base_url
    url = base_url + teacher_prefix + str(teacher_id) + page_prefix + str(page)
    return url

In [ ]:
review_list = []
for teacher_id in teacher_id_list:
    for page in range(1,page_range + 1):
        print("curr page:", page)
        # 너무 자주보내면 응답을 안 줌
        time.sleep(3)
        response = requests.get(getFullUrl(teacher_id=teacher_id, page=page))
        # 응답 성공
        if response.status_code == 200:
            # 한글 깨짐 방지 인코딩
            response.encoding = 'euc-kr'
            soup = BeautifulSoup(response.text, 'html.parser')
            # 첫 페이지면 선생님 이름 가져오기
            if page == 1:
                teacher_name = soup.select_one("div#TeacherLeftGnbEach a.tit").get_text(strip=True)
                print(teacher_name)
            # 리뷰가 적힌 행들 가져오기
            rows = soup.select('tbody > tr')
            for row in rows:
                # 등록 번호 찾기
                reg_num = row.select_one('td:nth-child(1)').get_text(strip=True)
                # BEST 수강 후기는 매 페이지마다 표시되므로 스킵
                if reg_num == "BEST": continue
                # 리뷰 내용 찾기
                review_text = row.select_one('td.wr_tit p.tit')
                ## 쓸데없는 정보 제거( new! 표시 )
                if review_text.span != None:
                    review_text.span.decompose()
                review_text = review_text.text.strip()
                # 등록 날짜 찾기
                date = row.select_one('td:nth-child(5)').get_text(strip=True)
                # 한 행 출력
                print(teacher_name, reg_num, review_text, date) 
                # csv로 저장할 행 생성
                review = [teacher_name, reg_num, review_text, date]
                # 전체 리뷰에 추가
                review_list.append(review)

In [17]:
# 파일로 저장
with open(csv_full_name, "a", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # 헤더 작성
    writer.writerow(["teacher_name", "no", "text", "date"])
    # 데이터 작성
    writer.writerows(review_list)